In [1]:
from gensim import corpora, models
dictionary = corpora.Dictionary.load('tmp/deerwester.dict')
corpus = corpora.MmCorpus('tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

C:\Users\user\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


MmCorpus(9 documents, 12 features, 28 non-zero entries)


Budujemy model LDA i transformujemy dane

In [2]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = model[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
for d in corpus_lda:
    print(d)

[(0, 0.19980547), (1, 0.80019456)]
[(0, 0.12004308), (1, 0.8799569)]
[(0, 0.86004144), (1, 0.13995859)]
[(0, 0.40960616), (1, 0.59039384)]
[(0, 0.7484901), (1, 0.2515099)]
[(0, 0.6543898), (1, 0.34561023)]
[(0, 0.58733881), (1, 0.41266119)]
[(0, 0.42098582), (1, 0.57901418)]
[(0, 0.1657773), (1, 0.83422273)]


<h1>Wizualizacja modeli</h1>

Do wizualizacji użyjemy biblioteki pyLDAvis. instalujemy pyLDAvis: https://github.com/bmabey/pyLDAvis

In [3]:
import pyLDAvis.gensim

vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)

ZADANIE: proszę wziąć 20newsgroups (dla każdej klasy 100 postów) i zwizualizować jego LDA. 

Czy znalezione tematy są łatwo opisywalne?

Czy podział na tematy zgadza się z podziałem na klasy?

In [4]:
#TODO
to_remove= ('headers', 'footers', 'quotes')

from sklearn.datasets import fetch_20newsgroups
twenty = fetch_20newsgroups(subset='all', shuffle=True)#, remove = to_remove)

In [5]:


print(len(twenty.data))
# 18846
 
print(len(twenty.target_names))
# 20
 
print(twenty.target_names)

18846
20
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [6]:
twenty.data[:5]

["From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n",
 'From: mblawson@midway.ecn.uoknor.edu (Matthew B Lawson)\nSubject: Which h

In [7]:
twenty.target[:5]

array([10,  3, 17,  3,  4])

In [8]:
#wyciagam dane z kolejnych klas
data = []
for j in range(len(twenty.target_names)):
    data.append([twenty.data[i] for i in range(twenty.target.shape[0]) if twenty.target[i] == j])

In [9]:
#biore okreslona ilosc danych z kazdej klasy

num=100
X_set = []
Y_set =[]
for i in range(20):
    X_set = X_set + data[i][:num]
    Y_set = Y_set + [i]*num

In [10]:
#wektoryzuje dane

from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(X_set)
print(dtm_tf.shape)

(2000, 3621)


In [13]:
#robie LDA ze sklearn

from sklearn.decomposition import LatentDirichletAllocation

# for TF DTM
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
# lda_tfidf.fit(dtm_tfidf)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=20,
             perp_tol=0.1, random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [14]:
#wizualizuje

import pyLDAvis.sklearn

vis = pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
pyLDAvis.display(vis)

Zastosowania: querowanie, rekomendacja, reprezentacja do klasyfiakcji

Na początek niech wezmą tutorial gensim, pobiorą dokument i stworzą reprezentację. Potem będziemy opowiadać o LDA

ZADANIA:

1. Mamy posty (tweety) użytkowników. Bierzemy jakiegoś użytkownika (zakładamy, że znamy dobrze jego zachowanie) i pytamy się, którzy użytkownicy są do niego podobni. Jeśli ich znajdziemy będziemy mogli im dedykować analogiczne reklamy i treści.

Rozw: można potraktować posty tego użytkownika jak query i wyszukać najbardziej podobne dokumenty innych użytkowników. Jeśli uzytkownicy mają dużo postów to trzeba jakąś miarę na zbiorach postów - dla prostoty porównujemy średnie, ale można też jakieś linkage functions z hierarchicznego wziąć.


2. Pytamy się o czym jest dany tekst. 

Rozw: Najprostsze rozwiązanie to znalezienie najczęstszych słów. Bardzej zaawansowane polega na wyszukaniu topiców (LSA albo LDA) - wtedy mamy niezależne topici.

3. Mamy podział na topici. Bierzemy użytkownika. Pytamy się którzy użytkownicy są do niego podobni biorąc pod uwagę tylko określony topic (na przykład pytamy sie którzy są podobni w kategorii sport. Zadanie możemy rozszerzyć na pytanie, którzy uyżytkownicy są do niego podobni biorąc pod uwagę kilka topiców na raz.

Rozw: Jeśli mamy topic albo topici to wystarczy zrzutować naszego użytkownika na tą podprzestrzeń. Tak samo robimy z innymi użytkownikami i szukamy najbliższych w tej podprzestrzeni.

4. Mamy podział na topici - które słowa charakteryzują najbardzie dany topic

Bierzemy te współrzędne (słowa) z największą normą z kolejnych wektorów własnych.

5. Jakie zdania najbardziej charakteryzują topic?

Jest to chyba opisane w https://towardsdatascience.com/nlp-for-topic-modeling-summarization-of-legal-documents-8c89393b1534 ale nie dokonca rozumiem. Jest tez notebook: https://github.com/chibueze07/Machine-Learning-In-Law/tree/master

6. Jakich osób (podmiotów) dotyczą topici. Trzeba zrobic POS i dla każdego podmiotu wyszukać podobieństwo do danego topicu. 


